In [1]:
from IPython.display import clear_output
import datetime
import numpy as np
import os
import pandas as pd
import re
import yfinance as yf

In [2]:
stock_list = pd.read_csv('raw/stock_list.csv')['Stock']
dtype_dict = {stock: 'float64' for stock in stock_list}
raw_feature_dict = pd.read_excel(os.getcwd() + '/raw/bbg_data.xlsm', sheet_name=None, dtype=dtype_dict, skiprows=4)
raw_feature_dict.update(pd.read_excel(os.getcwd() + '/raw/Bloomberg20032024data_raw.xlsx', sheet_name=None, dtype=dtype_dict, skiprows=4))
feature_list = list(raw_feature_dict.keys())

Merge the dataset with one time index

In [3]:
feature_dict = {}
for i in feature_list:
    df = raw_feature_dict[i]
    columns_list = list(df.columns)
    for num, col in enumerate(columns_list):
        if num % 2 == 0:
            columns_list[num] = 'Date'
    df.columns = columns_list
    df = df.astype({'Date':'datetime64[ns]'})
    df_merge = pd.merge(df.iloc[:, :2].dropna(), df.iloc[:, 2:4].dropna(), on='Date', how='outer')
    for j in range(len(columns_list) // 2 - 2):
        df_merge = df_merge.merge(df.iloc[:, 2 * j + 4:2 * j + 6].dropna(), on='Date', how='outer')
    feature_dict[i] = df_merge

Fill the missing values of each feature

In [4]:
for i in feature_list:
    # to do
    feature_df = feature_dict[i].set_index('Date').ffill().resample('ME').last().ffill()
    feature_dict[i] = feature_df.reset_index()

Remove stock if the missing proportion $> 0.05$

In [5]:
def missing_portion(df, start=(2012, 12, 31), end=(2023, 12, 31)):
    df = df[df['Date'].ge(datetime.datetime(*start)) & df['Date'].le(datetime.datetime(*end))]
    return np.sum(df.set_index('Date').isna(), axis=0) / df.shape[0]

In [6]:
removed_stock_list = []
for i in feature_list:
    # to do
    missing_series = missing_portion(feature_dict[i])
    removed_stock_list += list(missing_series[missing_series >= 0.02].index)
removed_stock_list = list(dict.fromkeys(removed_stock_list))

In [7]:
selected_stock_list = stock_list.copy()
for i in removed_stock_list:
    selected_stock_list.drop(selected_stock_list[selected_stock_list.eq(i)].index, inplace=True)

In [8]:
for i in feature_list:
    feature_dict[i] = feature_dict[i][pd.concat((pd.Series(['Date']), selected_stock_list))].dropna()

In [9]:
if not os.path.isdir('data'):
    os.mkdir('data')
    os.mkdir('data/price')
    os.mkdir('data/full')

In [10]:
start = '2012-12-31'
end = '2023-12-31'
price_dict = {}
for i in stock_list:
    code = re.sub(r'/', '-', i.split(' ')[0])
    price_df = yf.download(code, start=start, end=end, interval='1d')
    clear_output(wait=True)
    price_df.to_csv('data/price/{}_price.csv'.format(code))
    price_dict[i] = price_df

[*********************100%%**********************]  1 of 1 completed


In [11]:
code_list = selected_stock_list.apply(lambda x: re.sub(r'/', '-', x.split(' ')[0])).rename('Code')
pd.concat([selected_stock_list, code_list], axis=1).to_csv('data/selected_stock_list.csv', index=None)

stock_dict = {}
for num, i in enumerate(selected_stock_list):
    code = code_list.iloc[num]
    price_df = price_dict[i]
    price_df = pd.concat([price_df.resample('ME').first().iloc[:, 0], # open
                          price_df.resample('ME').max().iloc[:, 1], # high
                          price_df.resample('ME').min().iloc[:, 2], # low
                          price_df.resample('ME').last().iloc[:, 3:5], # close, adj close
                          price_df.resample('ME').mean().iloc[:, 5]], # volume
                         axis=1).reset_index()

    stock_df = feature_dict[feature_list[0]][['Date', i]].rename(columns={i: feature_list[0]})
    for j in feature_list[1:]:
        stock_df = stock_df.merge(feature_dict[j][['Date', i]].rename(columns={i: j}), on='Date', how='inner')

    stock_dict[i] = price_df.merge(stock_df, on='Date')
    stock_dict[i].set_index('Date').to_csv('data/full/{}_full.csv'.format(code))